In [1]:
pip install mediapipe keyboard deepFace

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:
import cv2
import mediapipe as mp
from deepface import DeepFace
import time

# Инициализация камеры
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Ширина
cap.set(4, 480)  # Высота
cap.set(10, 100)  # Яркость

# Инициализация MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7)

# Стиль для отрисовки landmarks
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=(0, 255, 0))

# Переменные для FPS
pTime = 0
emotion = "Neutral"
prev_emotion_time = 0
emotion_update_interval = 1  # Обновлять эмоцию каждую секунду

while True:
    success, img = cap.read()
    if not success:
        break
        
    img = cv2.flip(img, 1)  # Зеркальное отражение
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Обработка Face Mesh
    results = face_mesh.process(img_rgb)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Рисуем landmarks лица
            mp_drawing.draw_landmarks(
                image=img,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec)
            
            # Анализ эмоций с DeepFace (с ограниченной частотой)
            current_time = time.time()
            if current_time - prev_emotion_time > emotion_update_interval:
                try:
                    # Конвертируем в RGB для DeepFace
                    analysis = DeepFace.analyze(img_rgb, actions=['emotion'], enforce_detection=False)
                    if analysis and isinstance(analysis, list):
                        emotion = analysis[0]['dominant_emotion']
                    prev_emotion_time = current_time
                except Exception as e:
                    print(f"DeepFace error: {e}")
                    emotion = "Error"
    
    # Отображение информации
    cv2.putText(img, f'Emotion: {emotion}', (10, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Вычисление FPS
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (10, 80), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    
    cv2.imshow('Emotion Detection with DeepFace', img)
    if cv2.waitKey(1) == 27:  # Выход по ESC
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1744372322.723130 3911742 gl_context.cc:369] GL version: 2.1 (2.1 ATI-5.5.17), renderer: AMD Radeon Pro 570X OpenGL Engine
W0000 00:00:1744372322.726793 3928817 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744372322.742456 3928817 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


25-04-11 16:52:03 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /Users/student/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 7.99MB/s]
